# Collect data for the "flfa_acc" figure

- Save table to './results*plot/fig_flfa_acc*\*.csv'


In [1]:
import os
from pathlib import Path
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.lines import Line2D
from matplotlib.patches import Patch

from label_flip_revised.utils import create_dir


In [2]:
PATH_ROOT = Path(os.getcwd()).absolute().parent
print(PATH_ROOT)
PATH_OUTPUT = os.path.join(PATH_ROOT, "results_plot")
print(PATH_OUTPUT)
create_dir(PATH_OUTPUT)


/home/lukec/workspace/diva_01
/home/lukec/workspace/diva_01/results_plot


In [3]:
df_falfa_01 = pd.read_csv(os.path.join(PATH_ROOT, 'results', 'real', 'real_falfa_nn_db.csv'))
print(df_falfa_01.shape)
df_falfa_01.head()


(90, 38)


,Data,F1,F1 SD,F1v,F2,F3,F4,N1,N2,N2 SD,...,Hubs SD,Path.Train,Path.Poison,Path.Test,Rate,Train.Clean,Test.Clean,Train.Poison,Test.Poison,Data.Base
0,breastcancer_std_falfa_nn_0.00.csv,0.712464,0.215349,0.066235,6.264744e-11,0.485714,0.000000,0.074725,0.366983,0.074135,...,0.225796,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,0.00,0.986813,0.982456,0.986813,0.982456,breastcancer_std
1,breastcancer_std_falfa_nn_0.05.csv,0.859098,0.101038,0.133480,1.440449e-06,0.696703,0.000000,0.103297,0.382469,0.070786,...,0.283660,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,0.05,0.986813,0.982456,0.982418,0.956140,breastcancer_std
2,breastcancer_std_falfa_nn_0.10.csv,0.844907,0.108064,0.240749,2.910036e-09,0.753846,0.316484,0.191209,0.409176,0.076152,...,0.225717,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,0.10,0.986813,0.982456,0.964835,0.868421,breastcancer_std
3,breastcancer_std_falfa_nn_0.15.csv,0.948572,0.034941,0.258380,2.631059e-06,0.712088,0.048352,0.153846,0.396442,0.077667,...,0.181392,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,0.15,0.986813,0.982456,0.984615,0.824561,breastcancer_std
4,breastcancer_std_falfa_nn_0.20.csv,0.919837,0.052507,0.311281,6.912120e-10,0.723077,0.270330,0.217582,0.415749,0.075189,...,0.225626,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,/home/lukec/workspace/label_flip_revised_new/d...,0.20,0.986813,0.982456,0.971429,0.728070,breastcancer_std


In [4]:
datanames = np.sort(df_falfa_01['Data.Base'].unique())
print(datanames)


['abalone_subset_std' 'australian_std' 'banknote_std' 'breastcancer_std'
 'cmc_std' 'htru2_subset_std' 'phoneme_subset_std' 'ringnorm_subset_std'
 'texture_subset_std' 'yeast_subset_std']


In [5]:
workspace_names = [f'diva_{i:02d}' for i in range(1, 6)]
workspace_names


['diva_01', 'diva_02', 'diva_03', 'diva_04', 'diva_05']

In [6]:
ATTACKS = ['rand_svm', 'alfa_svm', 'poison_svm', 'falfa_nn']
ATTACKS


['rand_svm', 'alfa_svm', 'poison_svm', 'falfa_nn']

In [7]:
score_dict = {}
for dname in ATTACKS:
    dfs = []
    for wname in workspace_names:
        df_ = pd.read_csv(os.path.join(PATH_ROOT.parent, wname, 'results', 'real', f'real_{dname}_db.csv'))
        df_ = df_[['Data.Base', 'Rate', 'Train.Clean', 'Test.Clean', 'Train.Poison', 'Test.Poison']]
        dfs.append(df_)
    df = pd.concat(dfs, ignore_index=True).groupby(['Data.Base', 'Rate']).mean()
    path_result = os.path.join(PATH_OUTPUT, f'fig_flfa_acc_{dname}.csv')
    df.to_csv(path_result)
    print('Save to:', path_result)


Save to: /home/lukec/workspace/diva_01/results_plot/fig_flfa_acc_rand_svm.csv
Save to: /home/lukec/workspace/diva_01/results_plot/fig_flfa_acc_alfa_svm.csv
Save to: /home/lukec/workspace/diva_01/results_plot/fig_flfa_acc_poison_svm.csv
Save to: /home/lukec/workspace/diva_01/results_plot/fig_flfa_acc_falfa_nn.csv


In [8]:
datanames = ['abalone_subset_std', 'australian_std', 'banknote_std', 'breastcancer_std',
             'cmc_std', 'htru2_subset_std', 'phoneme_subset_std', 'ringnorm_subset_std',
             'texture_subset_std', 'yeast_subset_std']


In [9]:
# Load results
df_falfa = pd.read_csv(os.path.join(PATH_OUTPUT, f'fig_flfa_acc_falfa_nn.csv'))
df_rand = pd.read_csv(os.path.join(PATH_OUTPUT, f'fig_flfa_acc_rand_svm.csv'))
df_pois = pd.read_csv(os.path.join(PATH_OUTPUT, f'fig_flfa_acc_poison_svm.csv'))
df_alfa = pd.read_csv(os.path.join(PATH_OUTPUT, f'fig_flfa_acc_alfa_svm.csv'))

df_falfa.head()


,Data.Base,Rate,Train.Clean,Test.Clean,Train.Poison,Test.Poison
0,abalone_subset_std,0.00,0.789125,0.7745,0.789125,0.7745
1,abalone_subset_std,0.05,0.789125,0.7745,0.780750,0.7560
2,abalone_subset_std,0.10,0.789125,0.7745,0.697000,0.6975
3,abalone_subset_std,0.15,0.789125,0.7745,0.792750,0.6400
4,abalone_subset_std,0.20,0.789125,0.7745,0.752500,0.6045


In [10]:
SELECTED_DATA = [
    'texture_subset_std',
    'australian_std',
    'yeast_subset_std',
]


In [11]:
def create_fig_data(dfs, dataname):
    df_fig = pd.DataFrame()
    for i in range(len(dfs)):
        df_ = dfs[i]
        df_.loc[:, 'Attack'] = ATTACKS[i]
        df_ = df_.loc[:, ('Attack', 'Rate', 'Test.Poison', 'Train.Poison')]
        df_ = df_.melt(id_vars=['Attack', 'Rate'], var_name='Set', value_name='Accuracy')
        df_fig = pd.concat([df_fig, df_], ignore_index=True)
    return df_fig

In [12]:
df_figs = []
for dataname in SELECTED_DATA:
    dfs = [
        df_rand[df_rand['Data.Base'] == dataname].copy(),
        df_alfa[df_alfa['Data.Base'] == dataname].copy(),
        df_pois[df_pois['Data.Base'] == dataname].copy(),
        df_falfa[df_falfa['Data.Base'] == dataname].copy(),
    ]
    df_fig = create_fig_data(dfs, dataname)
    df_figs.append(df_fig)


In [13]:
file_names = ['texture', 'australian', 'yeast']
for _df, file_name in zip(df_figs, file_names):
    path_fig_data = os.path.join(PATH_OUTPUT, f'fig_flfa_acc_{file_name}.csv')
    _df.to_csv(path_fig_data, index=False)
    print('Save to:', path_fig_data)

Save to: /home/lukec/workspace/diva_01/results_plot/fig_flfa_acc_texture.csv
Save to: /home/lukec/workspace/diva_01/results_plot/fig_flfa_acc_australian.csv
Save to: /home/lukec/workspace/diva_01/results_plot/fig_flfa_acc_yeast.csv
